In [ ]:
pip install -U pip setuptools wheel

In [ ]:
!pip install -U pip setuptools wheel
!pip install -U spacy
!python -m spacy download en_core_web_sm
!python -m spacy download pt_core_news_md

In [ ]:

!python -m spacy download pt_core_news_md

In [ ]:
import spacy
spacy.load("pt_core_news_md")

In [ ]:
!python -m spacy validate

In [41]:
import spacy
from spacy.matcher import Matcher

nlp = spacy.load("pt_core_news_md")

def interpretar_mensagem(texto):
    doc = nlp(texto)
    matcher = Matcher(nlp.vocab)

    padrao_com_simbolo = [
        {"TEXT": {"REGEX": r"(?i)^r\$"}},
        {"LIKE_NUM": True}
    ]

    padrao_sem_simbolo = [
        {"LIKE_NUM": True},
        {"LOWER": "reais"}
    ]

    padrao_reais_centavos = [
        {"LIKE_NUM": True},
        {"LOWER": "reais"},
        {"LOWER": "e"},
        {"LIKE_NUM": True},
        {"LOWER": "centavos"}
    ]

    padrao_centavos = [
        {"LIKE_NUM": True},
        {"LOWER": "centavos"}
    ]

    matcher.add("VALOR", [
        padrao_reais_centavos,  # Priorizar combinados
        padrao_com_simbolo,
        padrao_sem_simbolo,
        padrao_centavos
    ])

    matches = matcher(doc)

    resultado = {"valor": 0.0}
    spans_usados = set()  # Para rastrear spans já processados

    for match_id, start, end in matches:
        span = doc[start:end]

        # Evitar duplicações: se já usamos esse span, pula
        if any(i in spans_usados for i in range(start, end)):
            continue

        tokens = [token.text.lower() for token in span]

        if tokens == ["r$", "200,50"] or tokens == ["r$", "200.50"]:
            try:
                resultado["valor"] += float(span[1].text.replace(",", "."))
                spans_usados.update(range(start, end))
            except:
                continue

        elif "reais" in tokens and "centavos" in tokens and "e" in tokens:
            try:
                reais_index = tokens.index("reais")
                centavos_index = tokens.index("centavos")
                reais = float(span[reais_index - 1].text.replace(",", "."))
                centavos = float(span[centavos_index - 1].text.replace(",", "."))
                resultado["valor"] += reais + (centavos / 100)
                spans_usados.update(range(start, end))
            except:
                continue

        elif len(tokens) == 2 and tokens[1] == "reais":
            try:
                resultado["valor"] += float(span[0].text.replace(",", "."))
                spans_usados.update(range(start, end))
            except:
                continue

        elif len(tokens) == 2 and tokens[1] == "centavos":
            try:
                resultado["valor"] += float(span[0].text.replace(",", ".")) / 100
                spans_usados.update(range(start, end))
            except:
                continue

        elif len(tokens) == 2 and "r$" in tokens[0]:
            try:
                resultado["valor"] += float(span[1].text.replace(",", "."))
                spans_usados.update(range(start, end))
            except:
                continue

    if resultado["valor"] == 0:
        resultado["valor"] = None
    else:
        resultado["valor"] = round(resultado["valor"], 2)

    return resultado


In [42]:
print(interpretar_mensagem("Hoje vendi R$ 500 de salgados para a escola."))
# {'valor': 500.0}

print(interpretar_mensagem("Hoje vendi r$ 200,50 de salgados para a escola."))
# {'valor': 200.5}

print(interpretar_mensagem("Hoje vendi 200 reais de salgados para a escola."))
# {'valor': 200.0}

print(interpretar_mensagem("Hoje vendi 500 reais e 50 centavos de salgados para a escola."))
# {'valor': 500.5}

print(interpretar_mensagem("Hoje vendi 50 centavos de salgados para a escola."))
# {'valor': 0.5}


{'valor': 500.0}
{'valor': 200.5}
{'valor': 200.0}
{'valor': 500.5}
{'valor': 0.5}
